In [ ]:
import time, random
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from opensky_api import OpenSkyApi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import smopy
#from scipy.misc import imresize

In [ ]:
%matplotlib

In [ ]:
def lat_lon_to_dist(lat_min, lon_min, lat_max, lon_max):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat_min)
    lon1 = radians(lon_min)
    lat2 = radians(lat_max)
    lon2 = radians(lon_max)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c * 1000

In [ ]:
# Prague, (Morina - Sojovice)
lat_min, lon_min = 49.951508, 14.212285
lat_max, lon_max = 50.221933, 14.763462
lat_center, lon_center = 50.101770, 14.263117
name = "Vaclav Havel's Airport"

# Vaclav Havel's Airport
lat_min, lon_min = 50.074975, 14.199871
lat_max, lon_max= 50.130103, 14.324053
lat_center, lon_center = 50.101770, 14.263117
name = "Vaclav Havel's Airport"

# London Heathrow Airport (Sunningdale - Wembley)
lat_min, lon_min = 51.392666, -0.633652
lat_max, lon_max = 51.550291, -0.301164
lat_center, lon_center = 51.467612, -0.453609
airport_name = "London Heathrow Airport"

# Chicago O'Hare International Airport (Wheaton - somewhere in the Michigan lake)
lat_min, lon_min = 41.868163, -88.097664
lat_max, lon_max = 42.163555, -87.601343
lat_center, lon_center = 41.980317, -87.912309
airport_name = "Chicago O'Hare International Airport"
"""
# Chicago O'Hare International Airport (Addison - Park Ridge)
lat_min, lon_min = 41.931889, -87.987572
lat_max, lon_max = 42.011809, -87.834858
lat_center, lon_center = 41.980317, -87.912309
airport_name = "Chicago O'Hare International Airport"
#"""
dlon = (lon_max - lon_min)
dlat = (lat_max - lat_min)
ratio = dlon / dlat
api = OpenSkyApi()

In [ ]:
s, line = None, None
anns, lst = [], []
d, color_dict = {}, {}
i, c = 0, 8

# Initialize figure.
fig = plt.figure(figsize=(c * ratio, c))
ax = fig.add_subplot(111, projection='3d')

#mp = smopy.Map((lat_min, lon_min, lat_max, lon_max))
#img = mp.to_numpy() / 255.0
#x, y = np.ogrid[0:img.shape[0], 0:img.shape[1]]
#ax.plot_surface(x, y, np.zeros(y.shape), rstride=5, cstride=5, facecolors=img)

#ax.set_xlim(lon_min, lon_max)
#ax.set_ylim(lat_min, lat_max)
ax.set_zlim(0, 2500)

#aa, bb = mp.to_pixels(lat_center, lon_center)
#aa, bb = lat_center, lon_center
bb = lat_lon_to_dist(lat_min, lon_min, lat_min, lon_center)
aa = lat_lon_to_dist(lat_min, lon_min, lat_center, lon_min)
ax.plot([bb], [aa], [0], 'o', c='green', markersize=16, label = airport_name)
plt.legend()

# Loop to the infinity.
while True:
    # Get flights data from API. Updated aproximatelly every 10-th sec.
    i = 0
    start = time.time()
    while True:
        if i > 0:
            lgnd = int((time.time() - start) / 1)
            t = ax.text2D(0.05, 0.95, str(lgnd), transform=ax.transAxes, fontsize=14, verticalalignment='top')
            plt.pause(1)
            t.remove()
        i += 1
        s2 = api.get_states(bbox = (lat_min, lat_max, lon_min, lon_max))
        if s2 is None: continue
        if s is None: pass
        elif s2.time == s.time: continue
        s = s2
        break
    lst.append({})
    # Process data.
    for state in s.states:
        # Filter unnamed flights and flights with None altitude.
        if len(state.callsign) < 1: continue
        #lat, lon = mp.to_pixels(state.latitude, state.longitude)
        #lat, lon = state.latitude, state.longitude
        lon = lat_lon_to_dist(lat_min, lon_min, lat_min, state.longitude)
        lat = lat_lon_to_dist(lat_min, lon_min, state.latitude, lon_min)
        if state.on_ground:
            lst[-1][state.callsign] = (lon, lat, 0)
        else:
            if state.baro_altitude is None: continue
            lst[-1][state.callsign] = (lon, lat, state.baro_altitude)

    # Load data for annotations.
    anns_info = []
    for flight in lst[-1]:
        if flight not in color_dict:
            color_dict[flight] = np.random.rand(3,)
        lon, lat, alt = lst[-1][flight]
        anns_info.append((flight, lon, lat, alt))
    # Delete old annotations from the figure.
    while len(anns) > 0:
        anns.pop().remove()
    
    # Plot figure.
    if len(lst) == 1:
        # Special case of the first iteration.
        for flight in lst[-1]:
            line, = ax.plot([lst[-1][flight][0]], [lst[-1][flight][1]], [lst[-1][flight][2]], 'x', c='black')
        for item in anns_info:
            ann = ax.text(item[1], item[2], item[3], item[0])
            anns.append(ann)
    else:
        # All other iterations.
        for flight in lst[-1]:
            if flight in lst[-2]:
                ax.plot([lst[-2][flight][0], lst[-1][flight][0]], [lst[-2][flight][1], lst[-1][flight][1]], [lst[-2][flight][2], lst[-1][flight][2]], '-x', c=color_dict[flight])
        for item in anns_info:
            ann = ax.text(item[1], item[2], item[3], item[0])
            anns.append(ann)
        plt.draw()
    plt.legend()